# Classificação - Presença de Doença Cardíaca

Este notebook realiza um estudo, um conjunto de experimentos, sobre algoritmos de classificação no dataset [Heart Disease UCI](https://www.kaggle.com/ronitf/heart-disease-uci). Um conjunto de dados que reúne mais de 300 pacientes com 14 atributos, tais como idade, sexo, nível do colesterol, nível de açucar no sangue, entre outros. Nosso objetivo é distinguir a presença (valor 1) ou ausência (valor 0) de uma doença cardíaca.

> Conteúdo voltado para iniciantes na área de Aprendizado de Máquina e Ciência de Dados!


<a id="top"></a>

## Conteúdo

> **Nota.** Alguns códigos foram ocultados a fim de facilitar a leitura.

O notebook está organizado como segue:

- [Dados](#data) - Carregamento dos dados, pré-processamento.
- [Visualização](#visual) - Análise exploratória dos dados.
- [Classificação](#class) - Aplicação de algoritmos de Aprendizado de Máquina.
    - [KNN](#knn) - Classificação com k-NN.
    - [Naive Bayes](#naive) - Classificação com Naive Bayes.
    - [Support Vector Machines](#svm) - Classificação com Support Vector Machines.
    - [Árvore de Decisão](#decision) - Classificação com Decision Tree.
    - [Random Forest](#forest) - Classificação com Random Forest.
    - [Bagging](#bagging) - Classificação com estratégia de Bagging.
    - [Ensemble](#ensemble) - Classificação com estratégia de Ensemble.
    - [AutoML](#automl) - Classificação usando Automated Machine Learning.
- [Hyperparameter Tuning](#tuning) - Tuning de parâmetros.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install auto-sklearn==0.12.0
!pip install scikit-learn==0.23.2

<a id="data"></a>

-----

# Dados

- Carregamento dos dados.
- Pré-processamento dos dados.

[Voltar para o Topo](#top)

## Carregamento dos Dados

In [ ]:
# algebra linear
import numpy as np 
# processamento de dados
import pandas as pd

In [ ]:
# imprime os arquvios
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/heart-disease-uci/heart.csv')
df.sample(3)

## Pré-Processamento dos Dados

Vamos normalizar os dados entre valor 0 e 1, utilizando o transformador `MinMaxScaler`. 
Este transformador normaliza os valores por coluna, utilizando o valor máximo e mínimo para normalizar o valor real.

> Este procedimento é necessário, pois alguns algoritmos de classificação se beneficiam de valores normalizados, tal como o K-NN.

In [ ]:
# normalizador
from sklearn import preprocessing

Segmenta os dados e as classes.

In [ ]:
# recupera os valores (X), e as classes (Y)
X = df.drop('target', axis=1)
Y = df['target']

### Normalização dos Dados

Nesta seção vamos utilizar a normalização [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html?highlight=minmaxscaler#sklearn.preprocessing.MinMaxScaler). Esta função de preprocessamento normaliza os dados conforme segue:

$$x_{new} = x_{\sigma} * (x_{max} - x_{min}) + x_{min}{}$$

Em que:

$$x_{\sigma} = \frac{(x - x_{min})}{(x_{max} - x_{min})}$$


A grosso modo, normaliza os dados entre 0 e 1 e mantem sua distribuição original.

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)

<a id="visual"></a>

-----

# Visualização

Nesta seção será realizado a transformação dos 13 atributos (removendo a classe) para 2 dimensões utilizando um algortimos de redução de dimensionadade, chamado de [PCA - Principal Component Analysis](https://en.wikipedia.org/wiki/Principal_component_analysis). 

> Faremos este procedimento a fim de visualizar os dados em 2 dimensões.

[Voltar para o Topo](#top)

In [ ]:
# visualização de dados
import seaborn as sns
import matplotlib.pyplot as plt

# redutor de dimensionalidade
from sklearn.decomposition import PCA

In [ ]:
# redução da dimensionalidade
pca = PCA(n_components=2).fit(X)
X_reduced = pca.fit_transform(X)

# registrando os valores num novo DataFrame
df_2d = pd.DataFrame(X_reduced)
df_2d.columns = ['0', '1']

Visualização dos Dados transformados em duas dimensões.

> **Note**. Os dados transformados não representam a realidade, ou seja, eles não são linearmente divididos como demonstrado visualmente. Contudo, indica que é possível realizar uma divisão nos dados.

In [ ]:
sns.scatterplot(data=df_2d, x='0', y='1')
plt.show()

Visualizando os dados transformados, colorindo pela classe.

In [ ]:
sns.scatterplot(data=df_2d, x='0', y='1', hue=Y, style=Y)
plt.show()

<a id="class"></a>

-----

# Classificação

- Conjunto de dados.
- Experimentos.

[Voltar para o Topo](#top)

In [ ]:
# métricas
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# variável de resultado final
# será armazenado o resultado de todos experimentos
experiment = {}

## Conjunto de Dados

Separa os conjuntos de treinamento e teste.

In [ ]:
# treinamento, test split
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=26)

In [ ]:
print('treinamento:', len(y_train))
print('teste      :', len(y_test))

<a id="knn"></a>

## K-NN

_(k-Nearest Neighbors)_

In [ ]:
# classificador
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
model1 = KNeighborsClassifier(n_neighbors=3)
model1.fit(X_train, y_train)

### Avaliação

In [ ]:
y_pred = model1.predict(X_test)

In [ ]:
acc = accuracy_score(y_pred, y_test)
pre = precision_score(y_pred, y_test)
rec = recall_score(y_pred, y_test)
f1  = f1_score(y_pred, y_test)

In [ ]:
experiment['KNN'] = {'acc':acc, 'pre':pre, 'rec':rec, 'f1':f1}

print('acurácia :', acc)
print('precisão :', pre)
print('revocação:', rec)
print('f1-score :', f1)

**Discussão k-NN**

O k-NN conseguiu classificar bem o conjunto de dados, alcançando resultados satisfatórios.   
Acurácia de 79% e F1-score de 82%.

-----

<a id="naive"></a>

## Naive Bayes

In [ ]:
# classificador
from sklearn.naive_bayes import GaussianNB

In [ ]:
model2 = GaussianNB()
model2.fit(X_train, y_train)

### Avaliação

In [ ]:
y_pred = model2.predict(X_test)

In [ ]:
acc = accuracy_score(y_pred, y_test)
pre = precision_score(y_pred, y_test)
rec = recall_score(y_pred, y_test)
f1  = f1_score(y_pred, y_test)

In [ ]:
experiment['Naive Bayes'] = {'acc':acc, 'pre':pre, 'rec':rec, 'f1':f1}

print('acurácia :', acc)
print('precisão :', pre)
print('revocação:', rec)
print('f1-score :', f1)

**Discussão Naive Bayes**

Neste conjunto de dados, o Naive Bayes teve um desempenho inferior ao k-NN.   
Contudo, apresentou uma acurácia de 74% e F1-Score de 77%.

-----

<a id="svm"></a>

## Support Vector Machines (SVM)

In [ ]:
# classificador
from sklearn.svm import SVC

In [ ]:
model3 = SVC()
model3.fit(X_train, y_train)

### Avaliação

In [ ]:
y_pred = model3.predict(X_test)

In [ ]:
acc = accuracy_score(y_pred, y_test)
pre = precision_score(y_pred, y_test)
rec = recall_score(y_pred, y_test)
f1  = f1_score(y_pred, y_test)

In [ ]:
experiment['SVM'] = {'acc':acc, 'pre':pre, 'rec':rec, 'f1':f1}

print('acurácia :', acc)
print('precisão :', pre)
print('revocação:', rec)
print('f1-score :', f1)

**Discussão Support Vector Machines (SVM)**

O SVM foi o melhor modelo até o momento.   
Ultrapassando o k-NN, com acurácia de 79% e F1-score de 82%.

-----

<a id="decision"></a>

## Árvore de Decisão

In [ ]:
# classificador
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model4 = DecisionTreeClassifier(random_state=26)
model4.fit(X_train, y_train)

### Avaliação

In [ ]:
y_pred = model4.predict(X_test)

In [ ]:
acc = accuracy_score(y_pred, y_test)
pre = precision_score(y_pred, y_test)
rec = recall_score(y_pred, y_test)
f1  = f1_score(y_pred, y_test)

In [ ]:
experiment['Decision Tree'] = {'acc':acc, 'pre':pre, 'rec':rec, 'f1':f1}

print('acurácia :', acc)
print('precisão :', pre)
print('revocação:', rec)
print('f1-score :', f1)

### Visualização

Nós conseguimos visualizar a árvore de decisão, como as ramificações ocorreram.   
É muito útil para uma apresentação de negócio, em que você consegue explicar a inteligência induzida.

In [ ]:
# visualizador da árvore
from sklearn.tree import plot_tree

Visualizando a árvore inteira.

In [ ]:
plt.figure(figsize=(10,6))
_ = plot_tree(model4) 
plt.show()

Visualizando a primeira profundidade, apenas para observarmos os valores presentes na figura.

> Na figura vemos: (1) O atributo selecionado e a questão (condição de separação) (nota, este nome pode ser personalizado); (2) a métrica de impureza; (3) número de exemplos; (4) número de exemplo para cada classe; e (5) a cor significa a classe majoritária do respectivo nó.

In [ ]:
plt.figure(figsize=(10,6))
_ = plot_tree(model4, max_depth=1) 
plt.show()

**Discussão Árvore de Decisão**

A Árvore de Decisão obteve resultados razoáveis.   
Com acurácia de 68% e F1-score de 70%, abaixo do k-NN.   

São excelente algoritmos de Aprendizado de Máquina para compreensão/estudo do negócio.

-----

<a id="forest"></a>

## Random Forest

In [ ]:
# classificador
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model5 = RandomForestClassifier(n_estimators=100, random_state=26)
model5.fit(X_train, y_train)

### Avaliação

In [ ]:
y_pred = model5.predict(X_test)

In [ ]:
acc = accuracy_score(y_pred, y_test)
pre = precision_score(y_pred, y_test)
rec = recall_score(y_pred, y_test)
f1  = f1_score(y_pred, y_test)

In [ ]:
experiment['Random Forest'] = {'acc':acc, 'pre':pre, 'rec':rec, 'f1':f1}

print('acurácia :', acc)
print('precisão :', pre)
print('revocação:', rec)
print('f1-score :', f1)

### Visualização

Também é possível extrar as `DecisionTree`do `RandomForest` para visualização. Neste caso, é necessário acessar cada uma das árvores utilizando o comando `RandomForest.estimators_[indice]` e visualizar como demonstrado na seção da Árvore de Decisão.

**Discussão Random Forest**

Random Forest obteve um dos melhores resultados.   
Com acurácia de 75% e F1-score de 79%, abaixo do k-NN.   

Random Forests são um dos algoritmos mais utilizados em competições de Aprendizado de Máquina.

> **Nota**. Possui alto custo computacional, pois tem que treinar vários modelos.

-----

<a id="bagging"></a>

## Bagging

Classificação com estratégia de Bagging, com algoritmo base SVM.

In [ ]:
# ensemble
from sklearn.ensemble import BaggingClassifier

In [ ]:
model_base = SVC()
model6 = BaggingClassifier(base_estimator=model_base, n_estimators=10, random_state=26)
model6.fit(X_train, y_train)

### Avaliação

In [ ]:
y_pred = model6.predict(X_test)

In [ ]:
acc = accuracy_score(y_pred, y_test)
pre = precision_score(y_pred, y_test)
rec = recall_score(y_pred, y_test)
f1  = f1_score(y_pred, y_test)

In [ ]:
experiment['Bagging'] = {'acc':acc, 'pre':pre, 'rec':rec, 'f1':f1}

print('acurácia :', acc)
print('precisão :', pre)
print('revocação:', rec)
print('f1-score :', f1)

**Discussão Bagging**

Bagging obteve o melhor resultado, utilizando 10 modelos SVM.   
Com acurácia de 82% e F1-score de 84%.   

> **Nota**. Possui alto custo computacional, pois tem que treinar vários modelos.

-----

<a id="ensemble"></a>

## Ensemble

Classificação com estratégia de Ensemble, utilizando os algoritmos SVM e Random Forest.

In [ ]:
# ensemble
from sklearn.ensemble import VotingClassifier

In [ ]:
clf1 = SVC()
clf2 = RandomForestClassifier(n_estimators=100, random_state=26)
estimators=[('SVM', clf1), ('RandomForest', clf2)]

model7 = VotingClassifier(estimators=estimators, voting='hard')
model7.fit(X_train, y_train)

### Avaliação

In [ ]:
y_pred = model7.predict(X_test)

In [ ]:
acc = accuracy_score(y_pred, y_test)
pre = precision_score(y_pred, y_test)
rec = recall_score(y_pred, y_test)
f1  = f1_score(y_pred, y_test)

In [ ]:
experiment['Ensemble'] = {'acc':acc, 'pre':pre, 'rec':rec, 'f1':f1}

print('acurácia :', acc)
print('precisão :', pre)
print('revocação:', rec)
print('f1-score :', f1)

**Discussão Ensemble**

Ensemble obteve bons resultados, porém não o melhor.   
Com acurácia de 78% e F1-score de 80%.   

> **Nota**. Possui alto custo computacional, pois tem que treinar vários modelos.

-----

<a id="automl"></a>

## AutoML

Automated Machine Learning.

In [ ]:
# automl
import autosklearn.classification as autoclassifier

In [ ]:
automl = autoclassifier.AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    tmp_folder='/automl/tmp/',
    output_folder='/automl/output/',
)
automl.fit(X_train, y_train, dataset_name='heart-disease-uci')

### Avaliação

In [ ]:
y_pred = automl.predict(X_test)

In [ ]:
acc = accuracy_score(y_pred, y_test)
pre = precision_score(y_pred, y_test)
rec = recall_score(y_pred, y_test)
f1  = f1_score(y_pred, y_test)

In [ ]:
experiment['AutoML'] = {'acc':acc, 'pre':pre, 'rec':rec, 'f1':f1}

print('acurácia :', acc)
print('precisão :', pre)
print('revocação:', rec)
print('f1-score :', f1)

### Visualização

Podemos ver o modelo ou o ensemble de modelos utilizado no AutoML.

> Para isto, utilize o comando `automl.show_models()`.

In [ ]:
# output escondido, ficou muito grande
automl.show_models()

# Conclusão

Por fim, o melhor algoritmo foi o SVM com a mesma acurácia do KNN, mas com precisão e f1-score maiores.   
A estratégia de ensemble Bagging superou todos os classificadores, porém teve maior custo computacional.

In [ ]:
# palheta de cores
import seaborn as sns

In [ ]:
cm = sns.color_palette('Blues_r', as_cmap=True)
pd.DataFrame(experiment).T.style.background_gradient(subset=['acc', 'f1'], cmap=cm).highlight_max(axis=0)

<a id="tuning"></a>

-----

# Hyperparameter Tuning

[Random Search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) no algoritmo SVM.

[Voltar para o Topo](#top)

In [ ]:
# random search
from sklearn.model_selection import RandomizedSearchCV
# uniform distribution
from scipy.stats import uniform

### Hiper-parâmetros

Quais são os hiper-parâmetros do SVM?

In [ ]:
model = SVC()
model.get_params()

Escolhendo uma distribuição dos parâmetros, _i.e.,_ um espaço de busca.

In [ ]:
distributions = {
    'random_state':[26],
    'C':uniform(loc=1, scale=9),
    'kernel': ['rbf', 'sigmoid'],
}

### Busca / Tuning

Executando a tunagem de parâmetros pelo espaço pré-fixado, utilizando 10-fold cross validation e em no máximo 50 experimentos, bem como otimizando a acurácia.

In [ ]:
clf = RandomizedSearchCV(model, distributions, n_iter=50, random_state=26, refit='acc', cv=10)
search = clf.fit(X_train, y_train)

Quais foram os melhores parâmetros?

In [ ]:
search.best_params_

Qual foi a acurácia deste parâmetro no dataset de validação?

In [ ]:
search.best_score_

### Avaliação

Avaliando o melhor modelo no dataset de teste.

In [ ]:
y_pred = search.predict(X_test)

In [ ]:
acc = accuracy_score(y_pred, y_test)
pre = precision_score(y_pred, y_test)
rec = recall_score(y_pred, y_test)
f1  = f1_score(y_pred, y_test)

In [ ]:
experiment['Random Search'] = {'acc':acc, 'pre':pre, 'rec':rec, 'f1':f1}

print('acurácia :', acc)
print('precisão :', pre)
print('revocação:', rec)
print('f1-score :', f1)

### Comparando o modelo padrão com o Tuning

Será que o Hyperparameter Tuning melhorou os resultados do SVM?

In [ ]:
pd.DataFrame(experiment)[['SVM', 'Random Search']].T.style.highlight_max(axis=0)